<h4 style="text-align: right;">ФБ-23 Качмар Михайло</h4>
<h1 style="text-align: center;"><b>Лабораторна робота №4</b></h1>
<h3 style="text-align: center;">Засоби підготовки та аналізу даних</h3>
<h5 style="text-aligleftter;"><b>Хід роботи:</b</h5>

In [17]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

print("Setup Complete")

Setup Complete


<h1><b>Перший рівень завдання</b></h1>

<h4>Завантаження датасету:</h4>

In [18]:
def download_first_task():

    download_url = 'https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip'
    file_name = 'individual+household+electric+power+consumption.zip'
    download_dir = 'first_task'
    os.makedirs(download_dir, exist_ok=True)

    file_path = os.path.join(download_dir, file_name)

    if not os.path.exists(file_path):
        try:
            with urllib.request.urlopen(download_url) as url:
                with open(file_path, 'wb') as f:
                    f.write(url.read())
            print(f"{file_name} успішно завантажено!")
        except Exception as error:
            print(f"Помилка при завантаженні: {error}")
    else:
        print(f"{file_name} вже завантажено.")

    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(download_dir)
        extracted_files = zip_ref.namelist()

    print(f"Папка зберігання архіву: {os.path.abspath(download_dir)}")
    print(f"Файли, отримані з архіву:")
    for file in extracted_files:
        print(f"{file}")

download_first_task()

individual+household+electric+power+consumption.zip вже завантажено.
Папка зберігання архіву: c:\Users\rdk\Documents\lab_7\laba_seven\first_task
Файли, отримані з архіву:
household_power_consumption.txt


<h4>Створення функцій для створення pandas dataframe та numpy array (data cleaning)</h4>

In [19]:
def pd_create():
    url = "first_task/household_power_consumption.txt"
    df = pd.read_csv(url, sep=";", low_memory=False, na_values=['?'])
    df.dropna(inplace=True)
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    print("Pandas dataframe успішно створений!")

    return df

def np_create():
    url = "first_task/household_power_consumption.txt"
    np_array = np.genfromtxt(url, delimiter=';', skip_header=1, dtype=str)
    mask = ~(np.any(np_array == '?', axis=1))
    np_array = np_array[mask]
    float_data = np_array[:,2:].astype(float)
    np_array = np.hstack([np_array[:, :2], float_data], dtype=object)
    print("Numpy array успішно створений!")

    return np_array

<h2><b>Pandas dataframe</b></h2>

In [20]:
import timeit
start_time = timeit.default_timer()
hd_power = pd_create()
end_time = timeit.default_timer()
pd_creating_time = end_time - start_time

hd_power.head()

Pandas dataframe успішно створений!


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


<h4>1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.</h4>

In [21]:
from timeit import timeit
def pddf_first_task(array):
    hds_over_5kw = array[array['Global_active_power'] > 5]
    return hds_over_5kw

first_task_pd = pddf_first_task(hd_power)
time_first_task_pd = timeit(lambda: pddf_first_task(hd_power), number=1)

print(f"Кількість домогосподарств з потужністю > 5 кВт: {len(first_task_pd)}")
print(f"Час виконання: {time_first_task_pd} секунд")
first_task_pd.head()


Кількість домогосподарств з потужністю > 5 кВт: 17547
Час виконання: 0.0053755000699311495 секунд


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
11,2006-12-16,17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
12,2006-12-16,17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0


<h4>2. Обрати всі домогосподарства, у яких вольтаж перевищує 235 В.</h4>

In [22]:
def pddf_second_task(array):
    hlds_over_235v = array[array['Voltage'] > 235]
    return hlds_over_235v

second_task_pd = pddf_second_task(hd_power)
time_second_task_pd = timeit(lambda: pddf_second_task(hd_power), number=1)

print(f"Кількість домогосподарств з вольтажем > 235 В: {len(second_task_pd)}")
print(f"Час виконання: {time_second_task_pd} секунд")
second_task_pd.head()

Кількість домогосподарств з вольтажем > 235 В: 1952491
Час виконання: 0.09090619999915361 секунд


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
5,2006-12-16,17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
6,2006-12-16,17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
7,2006-12-16,17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
14,2006-12-16,17:38:00,4.054,0.422,235.24,17.6,0.0,1.0,17.0


<h4>3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер.</h4>

In [23]:
def pddf_third_task(df):
    hds_19_20A = df.loc[(df['Global_intensity'] >= 19) & (df['Global_intensity'] <= 20)]
    hds_washer_fridge_gt_boiler_ac = hds_19_20A.loc[hds_19_20A['Sub_metering_2'] > hds_19_20A['Sub_metering_3']]
    return hds_washer_fridge_gt_boiler_ac

third_task_pd = pddf_third_task(hd_power)
time_third_task_pd = timeit(lambda: pddf_third_task(hd_power), number=1)

print(f"Кількість домогосподарств зі струмом 19-20А, де пральна машина та холодильник > бойлер і кондиціонер: {len(third_task_pd)}")
print(f"Час виконання: {time_third_task_pd} секунд")
third_task_pd.head()

Кількість домогосподарств зі струмом 19-20А, де пральна машина та холодильник > бойлер і кондиціонер: 2509
Час виконання: 0.00820049992762506 секунд


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
45,2006-12-16,18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0
460,2006-12-17,01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0
464,2006-12-17,01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0
475,2006-12-17,01:19:00,4.636,0.140,237.37,19.4,0.0,36.0,0.0
476,2006-12-17,01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0


<h4>4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії</h4>

In [24]:
def pddf_fourth_task(df):
    random_hds = df.sample(n=500000, replace=True)
    
    mean_sub_metering_1_pd = random_hds['Sub_metering_1'].mean()
    time_mean_sub_metering_1_pd = timeit(lambda: random_hds['Sub_metering_1'].mean(), number=1)
    print(f"Середнє споживання для групи 1: {mean_sub_metering_1_pd:.3f} Вт-годин")
    print(f"Час виконання для групи 1: {time_mean_sub_metering_1_pd} секунд\n")

    mean_sub_metering_2_pd = random_hds['Sub_metering_2'].mean()
    time_mean_sub_metering_2_pd = timeit(lambda: random_hds['Sub_metering_2'].mean(), number=1)
    print(f"Середнє споживання для групи 2: {mean_sub_metering_2_pd:.3f} Вт-годин")
    print(f"Час виконання для групи 2: {time_mean_sub_metering_2_pd} секунд\n")

    mean_sub_metering_3_pd = random_hds['Sub_metering_3'].mean()
    time_mean_sub_metering_3_pd = timeit(lambda: random_hds['Sub_metering_3'].mean(), number=1)
    print(f"Середнє споживання для групи 3: {mean_sub_metering_3_pd:.3f} Вт-годин")
    print(f"Час виконання для групи 3: {time_mean_sub_metering_3_pd} секунд")
    return time_mean_sub_metering_1_pd, time_mean_sub_metering_2_pd, time_mean_sub_metering_3_pd

time_mean_sub_metering_1_pd, time_mean_sub_metering_2_pd, time_mean_sub_metering_3_pd = pddf_fourth_task(hd_power)

Середнє споживання для групи 1: 1.125 Вт-годин
Час виконання для групи 1: 0.0008296000305563211 секунд

Середнє споживання для групи 2: 1.277 Вт-годин
Час виконання для групи 2: 0.0007298002019524574 секунд

Середнє споживання для групи 3: 6.466 Вт-годин
Час виконання для групи 3: 0.0008322999347001314 секунд


<h4>5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких:</h4>
<h5>a) основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), </h5>
<h5>b) а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини</h5>

In [25]:
def pddf_fifth_task(df):
    our_hours = datetime.strptime('18:00:00', '%H:%M:%S').time()
    df = df[df['Time'] > our_hours]
    df = df[df['Global_active_power'] > 6]

    df = df[df['Sub_metering_2'] > df['Sub_metering_1']]
    df = df[df['Sub_metering_2'] > df['Sub_metering_3']]

    n = len(df)

    first_half = df.iloc[:n//2]
    second_half = df.iloc[n//2:]
    selected_first_half = first_half.iloc[::3]
    selected_second_half = second_half.iloc[::4]
    selected_hds = pd.concat([selected_first_half, selected_second_half])

    return selected_hds


time_fifth_task_pd = timeit(lambda: pddf_fifth_task(hd_power), number=1)
result_task5np = pddf_fifth_task(hd_power)
print(f"Загальна кількість відповідних домогосподарств: {len(result_task5np)}")
print(f"Час виконання: {time_fifth_task_pd} секунд")
result_task5np

Загальна кількість відповідних домогосподарств: 310
Час виконання: 0.08015859988518059 секунд


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
41,2006-12-16,18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0
44,2006-12-16,18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0
17494,2006-12-28,20:58:00,6.386,0.374,236.63,27.0,1.0,36.0,17.0
17498,2006-12-28,21:02:00,8.088,0.262,235.50,34.4,1.0,72.0,17.0
17501,2006-12-28,21:05:00,7.230,0.152,235.22,30.6,1.0,73.0,17.0
...,...,...,...,...,...,...,...,...,...
2066467,2010-11-20,18:31:00,6.830,0.394,229.39,30.2,24.0,34.0,16.0
2066471,2010-11-20,18:35:00,6.784,0.364,228.38,30.0,21.0,35.0,16.0
2066475,2010-11-20,18:39:00,6.362,0.372,229.83,28.0,16.0,35.0,16.0
2066479,2010-11-20,18:43:00,6.324,0.376,229.93,27.8,14.0,35.0,17.0


<h2><b>Numpy array</b></h2>

In [26]:
import timeit
start_time = timeit.default_timer()
hd_power_array = np_create()
end_time = timeit.default_timer()
np_creating_time = end_time - start_time

print(hd_power_array)
print(f"Кількість записів: {len(hd_power_array)}")

Numpy array успішно створений!
[['16/12/2006' '17:24:00' 4.216 ... 0.0 1.0 17.0]
 ['16/12/2006' '17:25:00' 5.36 ... 0.0 1.0 16.0]
 ['16/12/2006' '17:26:00' 5.374 ... 0.0 2.0 17.0]
 ...
 ['26/11/2010' '21:00:00' 0.938 ... 0.0 0.0 0.0]
 ['26/11/2010' '21:01:00' 0.934 ... 0.0 0.0 0.0]
 ['26/11/2010' '21:02:00' 0.932 ... 0.0 0.0 0.0]]
Кількість записів: 2049280


<h4>1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.</h4>

In [27]:
from timeit import timeit
def npay_first_task(array):
    return array[array[:, 2] > 5]

first_task_np = npay_first_task(hd_power_array)
time_first_task_np = timeit(lambda: npay_first_task(hd_power_array), number=1)

print(f"Кількість домогосподарств з потужністю > 5 кВт: {len(first_task_np)}")
print(f"Час виконання: {time_first_task_np} секунд")
print(np.concatenate((first_task_np[:5], first_task_np[-5:])))

Кількість домогосподарств з потужністю > 5 кВт: 17547
Час виконання: 0.04832440009340644 секунд
[['16/12/2006' '17:25:00' 5.36 0.436 233.63 23.0 0.0 1.0 16.0]
 ['16/12/2006' '17:26:00' 5.374 0.498 233.29 23.0 0.0 2.0 17.0]
 ['16/12/2006' '17:27:00' 5.388 0.502 233.74 23.0 0.0 1.0 17.0]
 ['16/12/2006' '17:35:00' 5.412 0.47 232.78 23.2 0.0 1.0 17.0]
 ['16/12/2006' '17:36:00' 5.224 0.478 232.99 22.4 0.0 1.0 16.0]
 ['22/11/2010' '18:40:00' 5.408 0.15 231.5 23.6 48.0 0.0 0.0]
 ['22/11/2010' '18:41:00' 5.528 0.144 232.48 24.6 53.0 0.0 0.0]
 ['24/11/2010' '07:50:00' 5.172 0.05 235.18 22.0 0.0 38.0 17.0]
 ['24/11/2010' '07:51:00' 5.75 0.0 234.4 24.6 0.0 39.0 17.0]
 ['25/11/2010' '07:21:00' 5.074 0.24 238.55 21.4 1.0 2.0 18.0]]


<h4>2. Обрати всі домогосподарства, у яких вольтаж перевищує 235 В.</h4>

In [28]:
def npay_second_task(array):
    return array[array[:, 4] > 235]

second_task_np = npay_second_task(hd_power_array)
time_second_task_np = timeit(lambda: npay_second_task(hd_power_array), number=1)

print(f"Кількість домогосподарств з вольтажем > 235 В: {len(second_task_np)}")
print(f"Час виконання: {time_second_task_np} секунд")
print(np.concatenate((second_task_np[:5], second_task_np[-5:])))

Кількість домогосподарств з вольтажем > 235 В: 1952491
Час виконання: 0.2560325001832098 секунд
[['16/12/2006' '17:28:00' 3.666 0.528 235.68 15.8 0.0 1.0 17.0]
 ['16/12/2006' '17:29:00' 3.52 0.522 235.02 15.0 0.0 2.0 17.0]
 ['16/12/2006' '17:30:00' 3.702 0.52 235.09 15.8 0.0 1.0 17.0]
 ['16/12/2006' '17:31:00' 3.7 0.52 235.22 15.8 0.0 1.0 17.0]
 ['16/12/2006' '17:38:00' 4.054 0.422 235.24 17.6 0.0 1.0 17.0]
 ['26/11/2010' '20:58:00' 0.946 0.0 240.43 4.0 0.0 0.0 0.0]
 ['26/11/2010' '20:59:00' 0.944 0.0 240.0 4.0 0.0 0.0 0.0]
 ['26/11/2010' '21:00:00' 0.938 0.0 239.82 3.8 0.0 0.0 0.0]
 ['26/11/2010' '21:01:00' 0.934 0.0 239.7 3.8 0.0 0.0 0.0]
 ['26/11/2010' '21:02:00' 0.932 0.0 239.55 3.8 0.0 0.0 0.0]]


<h4>3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.</h4>

In [29]:
def npay_third_task(array):
    ampers_filtred = array[(array[:, 5] >= 19) & (array[:, 5] <= 20)]
    high_sub2_sub3_hds = ampers_filtred[ampers_filtred[:, 7] > ampers_filtred[:, 8]]
    return high_sub2_sub3_hds

third_task_np = npay_third_task(hd_power_array)
time_third_task_np = timeit(lambda: npay_third_task(hd_power_array), number=1)

print(f"Кількість домогосподарств з силою струму 19-20 А, де sub_metering_2 > sub_metering_3: {third_task_np.shape[0]}")
print(f"Час виконання: {time_third_task_np} секунд")
print(np.concatenate((third_task_np[:5], third_task_np[-5:])))

Кількість домогосподарств з силою струму 19-20 А, де sub_metering_2 > sub_metering_3: 2509
Час виконання: 0.10391970002092421 секунд
[['16/12/2006' '18:09:00' 4.464 0.136 234.66 19.0 0.0 37.0 16.0]
 ['17/12/2006' '01:04:00' 4.582 0.258 238.08 19.6 0.0 13.0 0.0]
 ['17/12/2006' '01:08:00' 4.618 0.104 239.61 19.6 0.0 27.0 0.0]
 ['17/12/2006' '01:19:00' 4.636 0.14 237.37 19.4 0.0 36.0 0.0]
 ['17/12/2006' '01:20:00' 4.634 0.152 237.17 19.4 0.0 35.0 0.0]
 ['24/11/2010' '07:53:00' 4.666 0.0 235.72 19.8 0.0 39.0 17.0]
 ['24/11/2010' '07:54:00' 4.694 0.0 236.78 19.8 0.0 39.0 18.0]
 ['24/11/2010' '07:55:00' 4.602 0.0 237.08 19.4 0.0 40.0 17.0]
 ['24/11/2010' '07:56:00' 4.536 0.0 237.03 19.0 0.0 39.0 17.0]
 ['24/11/2010' '07:57:00' 4.626 0.0 236.78 19.4 0.0 39.0 17.0]]


<h4>4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії</h4>

In [30]:
def npay_fourth_task(array):
    random_indices = np.random.choice(array.shape[0], size=500000, replace=True)
    random_hds = array[random_indices]

    mean_sub_metering_1_np = np.mean(random_hds[:, 6])
    time1 = timeit(lambda: np.mean(random_hds[:, 6]), number=1)
    print(f"Середнє споживання для групи 1: {mean_sub_metering_1_np:.3f} Вт-годин")
    print(f"Час виконання для групи 1: {time1} секунд\n")

    mean_sub_metering_2_np = np.mean(random_hds[:, 7]) 
    time2 = timeit(lambda: np.mean(random_hds[:, 7]), number=1)
    print(f"Середнє споживання для групи 2: {mean_sub_metering_2_np:.3f} Вт-годин")
    print(f"Час виконання для групи 1: {time2} секунд\n")

    mean_sub_metering_3_np = np.mean(random_hds[:, 8])
    time3 = timeit(lambda: np.mean(random_hds[:, 8]), number=1)
    print(f"Середнє споживання для групи 3: {mean_sub_metering_3_np:.3f} Вт-годин")
    print(f"Час виконання для групи 1: {time3} секунд\n")

    return time1, time2, time3
time_mean_sub_metering_1_np, time_mean_sub_metering_2_np, time_mean_sub_metering_3_np = npay_fourth_task(hd_power_array)

Середнє споживання для групи 1: 1.125 Вт-годин
Час виконання для групи 1: 0.0267232998739928 секунд

Середнє споживання для групи 2: 1.307 Вт-годин
Час виконання для групи 1: 0.026705800089985132 секунд

Середнє споживання для групи 3: 6.450 Вт-годин
Час виконання для групи 1: 0.025904100155457854 секунд



<h4>5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких:</h4>
<h5>a) основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), </h5>
<h5>b) а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини</h5>

In [31]:
def npay_fifth_task(df):
    df = df[(df[:, 1] >= '18:00:00') & (df[:, 2] > 6)]
    df = df[df[:, 7] > df[:, 6]]
    df = df[df[:, 7] > df[:, 8]]

    if df.shape[0] > 0:
        first_half = df[:len(df)//2]
        second_half = df[len(df)//2:]

        selected_first_half = first_half[::3]
        selected_second_half = second_half[::4]

        selected_hds = np.concatenate((selected_first_half, selected_second_half))
        return selected_hds
    else:
        print("Не знайдено домогосподарств, що задовольняють умови.")
        return None
    
time_fifth_task_np = timeit(lambda: npay_fifth_task(hd_power_array), number=1)  
fifth_task_np = npay_fifth_task(hd_power_array)
print(f"Кількість відібраних домогосподарств: {len(fifth_task_np)}")
print(np.concatenate((fifth_task_np[:5], fifth_task_np[-5:])))

Кількість відібраних домогосподарств: 310
[['16/12/2006' '18:05:00' 6.052 0.192 232.93 26.2 0.0 37.0 17.0]
 ['16/12/2006' '18:08:00' 6.308 0.116 232.25 27.0 0.0 36.0 17.0]
 ['28/12/2006' '20:58:00' 6.386 0.374 236.63 27.0 1.0 36.0 17.0]
 ['28/12/2006' '21:02:00' 8.088 0.262 235.5 34.4 1.0 72.0 17.0]
 ['28/12/2006' '21:05:00' 7.23 0.152 235.22 30.6 1.0 73.0 17.0]
 ['20/11/2010' '18:30:00' 6.62 0.314 229.0 29.2 18.0 34.0 16.0]
 ['20/11/2010' '18:34:00' 6.266 0.36 229.82 27.6 14.0 35.0 16.0]
 ['20/11/2010' '18:38:00' 6.302 0.376 229.65 27.8 15.0 34.0 17.0]
 ['20/11/2010' '18:42:00' 6.238 0.344 229.37 27.6 14.0 35.0 16.0]
 ['20/11/2010' '18:46:00' 6.438 0.308 229.26 28.4 13.0 39.0 16.0]]


<h4>Таблиця з часом виконання: </h4>

In [32]:
time_data = {
    'Час створення': [pd_creating_time, np_creating_time],
    'Завдання 1': [time_first_task_pd, time_first_task_np],
    'Завдання 2': [time_second_task_pd, time_second_task_np],
    'Завдання 3': [time_third_task_pd, time_third_task_np],
    'Завдання 4.1': [time_mean_sub_metering_1_pd, time_mean_sub_metering_1_np],
    'Завдання 4.2': [time_mean_sub_metering_2_pd, time_mean_sub_metering_2_np],
    'Завдання 4.3': [time_mean_sub_metering_3_pd, time_mean_sub_metering_3_np],
    'Завдання 5': [time_fifth_task_pd, time_fifth_task_np]
}
df = pd.DataFrame(time_data, index=['PD', 'NP'])
print(df)
print("У всіх завданнях Pandas швидший за Numpy.")


    Час створення  Завдання 1  Завдання 2  Завдання 3  Завдання 4.1  \
PD       6.772739    0.005376    0.090906     0.00820      0.000830   
NP      24.902895    0.048324    0.256033     0.10392      0.026723   

    Завдання 4.2  Завдання 4.3  Завдання 5  
PD      0.000730      0.000832    0.080159  
NP      0.026706      0.025904    0.106931  
У всіх завданнях Pandas швидший за Numpy.
